In [5]:
import dask.distributed
import pystac_client
import stackstac
import dask.array as da
from geogif import gif, dgif
import geopandas as gpd

#Load STAC items into xarray Datasets. Process locally or distribute data loading and computation with Dask.
#from odc.stac import configure_rio, load
from IPython.display import HTML, display
import folium
import folium.plugins
from branca.element import Figure
import shapely.geometry

In [6]:
client = dask.distributed.Client()
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 80,Total memory: 187.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41923,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 80
Started: Just now,Total memory: 187.56 GiB
Comm: tcp://127.0.0.1:36595,Total threads: 8
Dashboard: http://127.0.0.1:41895/status,Memory: 18.76 GiB
Nanny: tcp://127.0.0.1:38001,


In [7]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [8]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    aoi['geometry'],
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)


map1.fit_bounds(bounds=convert_bounds(aoi.unary_union.bounds))
display(fig)

With the pystac_client module’s Client class, Open the STAC API. 

In [19]:
%%time

bbox =aoi.unary_union.bounds
items = (
    pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    .search(
        bbox=bbox,
        collections=["sentinel-2-l2a"],
        query={"eo:cloud_cover":{"lt":20}},
        datetime = "2016-05-01/2022-06-30"
    )
)

items.matched()

CPU times: user 86.3 ms, sys: 8.15 ms, total: 94.4 ms
Wall time: 291 ms


Turn STAC items into xarray as a temporal stack, using stackstac.


In [20]:
stack = stackstac.stack(items.item_collection(), bounds_latlon=bbox, epsg = 32620, resolution=30)

/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/stackstac/accumulate_metadata.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(


In [8]:
type (stack)

xarray.core.dataarray.DataArray


Then mask out bad (cloudy) pixels, according to the Sentinel-2 SCL Scene Classification Map, and take the temporal median of each quarter (three months) to hopefully get an okay-looking cloud-free frame representative of those three months.

In [21]:

scl = stack.sel(band=["SCL"])
# Sentinel-2 Scene Classification Map: nodata, saturated/defective, dark, cloud shadow, cloud med. prob., cloud high prob., cirrus
invalid = da.isin(scl, [0, 1, 2, 3, 8, 9, 10])
valid = stack.where(~invalid)

rgb = valid.sel(band=["B04", "B03", "B02"])

quarterly = rgb.resample(time="Q").median()
quarterly

/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/dask/array/core.py:1760: PerformanceWarning: Increasing number of chunks by factor of 100
  return da_func(*args, **kwargs)
/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 57
  result = blockwise(
/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/dask/array/core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 272
  result = blockwise(


,Array,Chunk
Bytes,4.26 GiB,172.27 kiB
Shape,"(25, 3, 2696, 2826)","(2, 1, 105, 105)"
Count,124 Graph Layers,4015536 Chunks
Type,float64,numpy.ndarray


In [ ]:
#ts = quarterly.persist()
#ts_local = quarterly.compute()
#ts_local.plot.imshow(col="time", rgb="band", col_wrap=5, robust=True)

In [22]:
ts = quarterly.persist()
cleaned = ts[2:].ffill("time").bfill("time")

Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x7fef5a88afa0>>
Traceback (most recent call last):
  File "/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/distributed/utils_perf.py", line 179, in _gc_callback
    def _gc_callback(self, phase, info):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
gif_data = dgif(cleaned, bytes=True).compute()


In [ ]:
#dgif(cleaned).compute()


In [ ]:
with open("petermann_1sem2020.gif", "wb") as f:
    f.write(gif_data)